In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [2]:
# Tempory data frame to pull in the drinks.csv file
temp_df = pd.read_csv("output_data/drinks.csv")

In [3]:
# Display head of data frame. Feel free to delete
temp_df.head()

,Drink ID,Name,Category,Alcoholic,Glass,Ingredients,Measure
0,11029,Amaretto Tea,Ordinary Drink,True,Pousse cafe glass,"['Tea', 'Amaretto', 'Whipped cream']","['6 oz hot ', '2 oz ', 'Chilled ']"
1,13825,Midnight Manx,Ordinary Drink,True,Old-fashioned glass,"['Kahlua', 'Baileys irish cream', 'Goldschlage...","['1 oz ', '1 oz ', 'dash ', '2 oz ', '2 oz Haz..."
2,16405,A Piece of Ass,Other/Unknown,True,Highball glass,"['Amaretto', 'Southern Comfort', 'Ice', 'Sour ...","['1 shot ', '1 shot ', 'cubes', None]"
3,12954,Holloween Punch,Punch / Party Drink,False,Punch bowl,"['Grape juice', 'Carbonated soft drink', 'Sher...","[None, ', orange\n', None, None]"
4,11379,"French ""75""",Ordinary Drink,True,Collins glass,"['Gin', 'Sugar', 'Lemon juice', 'Champagne', '...","['1 1/2 oz ', '2 tsp superfine ', '1 1/2 oz ',..."


In [4]:
# List conversion function. Let me know if there are any issues
def convert(s):
    lst = []
    temp = s.split(", ")
    
    for x in temp:
        lst.append(x.translate({ord(i): None for i in "[']"}))
    
    return lst

In [5]:
# Necessary copy of data frame. The value count is just a test. Feel free to delete that line
drink_df = temp_df.copy()
drink_df['Glass'].value_counts()

Cocktail glass         26
Highball glass         11
Collins glass           8
Old-fashioned glass     6
Shot glass              5
Highball Glass          5
Coffee mug              4
Whiskey sour glass      3
Hurricane glass         3
Balloon Glass           3
Champagne flute         3
Punch bowl              3
Collins Glass           3
Nick and Nora Glass     2
Martini Glass           2
Pousse cafe glass       2
Mason jar               2
Beer mug                2
Cordial glass           1
Old-Fashioned glass     1
Beer pilsner            1
Irish coffee cup        1
Shot Glass              1
White wine glass        1
Name: Glass, dtype: int64

In [6]:
# Creation of main data frame. All plots should use this data (i.e. drink_df)
for i in range(0, len(drink_df)):
    drink_df.at[i,'Ingredients'] = convert(temp_df.loc[i,'Ingredients'])

In [7]:
# Creation of sorted list of tuples. Useful for most used ingredient
ing_count = {}
for i in range(0, len(drink_df)):
    for ing in drink_df.loc[i,'Ingredients']:
        if ing in ing_count:
            ing_count[ing] = ing_count[ing] + 1
        else:
            ing_count[ing] = 1
            
#print(ing_count)
a=sorted(ing_count.items(), key=lambda x: x[1], reverse = True)
print(a)
#print(drink2_df['Ingredients'].to_string())

[('Gin', 23), ('Vodka', 14), ('Lemon', 13), ('Amaretto', 12), ('Orange juice', 8), ('Lemon juice', 7), ('Light rum', 7), ('Lime', 7), ('Tequila', 7), ('Triple sec', 7), ('Kahlua', 6), ('Ice', 6), ('Sugar', 6), ('Powdered sugar', 6), ('Baileys irish cream', 5), ('Coffee', 5), ('Maraschino cherry', 5), ('Egg', 5), ('Light cream', 5), ('Lime Juice', 5), ('Dry Vermouth', 5), ('Maraschino liqueur', 5), ('Orange bitters', 5), ('Grenadine', 5), ('Milk', 5), ('Ginger', 5), ('Champagne', 4), ('Carbonated water', 4), ('Cranberry juice', 4), ('Ginger ale', 4), ('Sweet Vermouth', 4), ('151 proof rum', 4), ('Water', 4), ('Lime juice', 4), ('Whipped cream', 3), ('Orange', 3), ('Bitters', 3), ('Coca-Cola', 3), ('Blue Curacao', 3), ('Brandy', 3), ('Bourbon', 3), ('Triple Sec', 3), ('7-Up', 3), ('Creme de Cassis', 3), ('Angostura Bitters', 3), ('Chocolate', 3), ('Cinnamon', 3), ('Strawberries', 3), ('Cherry', 3), ('Absolut Citron', 3), ('Tea', 2), ('Goldschlager', 2), ('Heavy cream', 2), ('Sour mix', 2

In [8]:
# Creation of new series. Feel free to delete if not needed.

drink_df['Ingredients Count'] = 0
for i in range(0, len(drink_df)):
    drink_df.loc[i,'Ingredients Count'] = len(drink_df.loc[i,'Ingredients'])
drink_df.head()

,Drink ID,Name,Category,Alcoholic,Glass,Ingredients,Measure,Ingredients Count
0,11029,Amaretto Tea,Ordinary Drink,True,Pousse cafe glass,"[Tea, Amaretto, Whipped cream]","['6 oz hot ', '2 oz ', 'Chilled ']",3
1,13825,Midnight Manx,Ordinary Drink,True,Old-fashioned glass,"[Kahlua, Baileys irish cream, Goldschlager, He...","['1 oz ', '1 oz ', 'dash ', '2 oz ', '2 oz Haz...",5
2,16405,A Piece of Ass,Other/Unknown,True,Highball glass,"[Amaretto, Southern Comfort, Ice, Sour mix]","['1 shot ', '1 shot ', 'cubes', None]",4
3,12954,Holloween Punch,Punch / Party Drink,False,Punch bowl,"[Grape juice, Carbonated soft drink, Sherbet, ...","[None, ', orange\n', None, None]",4
4,11379,"French ""75""",Ordinary Drink,True,Collins glass,"[Gin, Sugar, Lemon juice, Champagne, Orange, M...","['1 1/2 oz ', '2 tsp superfine ', '1 1/2 oz ',...",6
